In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
import tensorflow_datasets as tfds
import numpy as np

# Load Yelp Polarity dataset
dataset, info = tfds.load('yelp_polarity_reviews', with_info=True, as_supervised=True)
train_data, test_data = dataset['train'], dataset['test']

# Prepare the data
vocab_size = 50000  # Limit vocabulary to top 50,000 words
max_length = 200    # Maximum review length (truncation/padding)

# Tokenizer to convert text to sequences
tokenizer = keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts([text.numpy().decode('utf-8') for text, _ in train_data])

# Convert text data to sequences and pad them
def preprocess_dataset(dataset):
    texts, labels = [], []
    for text, label in dataset:
        texts.append(text.numpy().decode('utf-8'))
        labels.append(label.numpy())
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
    return padded_sequences, np.array(labels)

# Preprocess training and testing data
tr_x, tr_y = preprocess_dataset(train_data)
te_x, te_y = preprocess_dataset(test_data)

# Build the model
model = keras.Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length),  # Embedding Layer
    Bidirectional(LSTM(32, return_sequences=True, kernel_regularizer=l2(0.001))),  # Bidirectional LSTM
    Dropout(0.5),  # Dropout to prevent overfitting
    GlobalAveragePooling1D(),  # Average pooling over time steps
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),  # Fully connected layer with L2 reg
    Dropout(0.5),  # Dropout before output layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Add early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model
model.fit(tr_x, tr_y, epochs=10, batch_size=64, validation_data=(te_x, te_y), callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(te_x, te_y)
print(f"Test Accuracy: {test_acc:.4f}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/560000 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/38000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/yelp_polarity_reviews/incomplete.1L9156_0.2.0/yelp_polarity_reviews-test.t…

Dataset yelp_polarity_reviews downloaded and prepared to /root/tensorflow_datasets/yelp_polarity_reviews/0.2.0. Subsequent calls will reuse this data.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
8750/8750 ━━━━━━━━━━━━━━━━━━━━ 186s 20ms/step - accuracy: 0.8674 - loss: 0.3491 - val_accuracy: 0.9246 - val_loss: 0.2447
Epoch 2/10
8750/8750 ━━━━━━━━━━━━━━━━━━━━ 199s 21ms/step - accuracy: 0.9389 - loss: 0.1835 - val_accuracy: 0.9466 - val_loss: 0.1584
Epoch 3/10
8750/8750 ━━━━━━━━━━━━━━━━━━━━ 200s 20ms/step - accuracy: 0.9528 - loss: 0.1465 - val_accuracy: 0.9465 - val_loss: 0.1530
Epoch 4/10
8750/8750 ━━━━━━━━━━━━━━━━━━━━ 202s 20ms/step - accuracy: 0.9603 - loss: 0.1247 - val_accuracy: 0.9488 - val_loss: 0.1516
Epoch 5/10
8750/8750 ━━━━━━━━━━━━━━━━━━━━ 203s 20ms/step - accuracy: 0.9655 - loss: 0.1118 - val_accuracy: 0.9482 - val_loss: 0.1509
Epoch 6/10
8750/8750 ━━━━━━━━━━━━━━━━━━━━ 201s 20ms/step - accuracy: 0.9696 - loss: 0.1003 - val_accuracy: 0.9431 - val_loss: 0.1799
Epoch 7/10
8750/8750 ━━━━━━━━━━━━━━━━━━━━ 204s 20ms/step - accuracy: 0.9734 - loss: 0.0907 - val_accuracy: 0.9464 - val_loss: 0.1662
1188/1188 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9473 - loss

In [3]:
# Save the entire model
model.save('yelp_sentiment_model.keras')

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to predict sentiment of input text
def predict_sentiment(model, text, tokenizer, max_length=200):
    # Convert text to sequence using the tokenizer
    sequence = tokenizer.texts_to_sequences([text])

    # Pad the sequence to the fixed length
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')

    # Get prediction probability
    prediction = model.predict(padded_sequence)[0][0]

    # Classify sentiment
    sentiment = "Positive" if prediction > 0.5 else "Negative"

    # Confidence score
    confidence = prediction if sentiment == "Positive" else 1 - prediction

    return sentiment, confidence
    # Function to decode a sequence back to text
def sequence_to_text(sequence, tokenizer):
    return tokenizer.sequences_to_texts([sequence])[0]

# Select random samples from the test set
num_samples = 5  # Number of samples to test
random_indices = np.random.choice(len(te_x), num_samples, replace=False)

# Test the selected samples
for i in random_indices:
    # Get the sequence and label
    sequence = te_x[i]
    true_label = te_y[i]

    # Decode the sequence to text
    review_text = sequence_to_text(sequence, tokenizer)

    # Predict sentiment using the model
    predicted_sentiment, confidence = predict_sentiment(model, review_text, tokenizer)

    # Map the true label to a sentiment
    true_sentiment = "Positive" if true_label == 1 else "Negative"

    # Print results
    print(f"Review: {review_text}")
    print(f"True Sentiment: {true_sentiment}")
    print(f"Predicted Sentiment: {predicted_sentiment} (Confidence: {confidence:.4f})")
    print("-" * 50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
Review: one of kiddo's friends had a birthday party up in anthem which is very far from where we live so rather than drive back home and come back to get him we decided to make a night out of it after we dropped him off at the party we drove around and spotted this place using the yelp app after browsing the menu we decided to order n nbrisket ribs combo platter w slaw fries 13 95 nhttp www yelp com biz photos q to u bbq anthem select nhttp www yelp com biz photos q to u bbq anthem select n npork ribs combo platter w beans slaw 13 95 nhttp www yelp com biz photos q to u bbq anthem select nhttp www yelp com biz photos q to u bbq anthem select nhttp www yelp com biz photos q to u bbq anthem select n nthe joint is pretty small and we were there during the peak dinner hour so it got packed luckily we found a table n nafter a little while the food came out we were starving so it had no chance to last very long i had the brisket ribs the brisket was
Tru

In [5]:
# List of positive and negative reviews to test
test_reviews = [
    # Positive reviews
    "The food was absolutely delicious, and the service was exceptional!",
    "I had a wonderful experience at this restaurant. The ambiance was perfect, and the staff was very friendly.",
    "Highly recommend this place! The dishes were flavorful, and the presentation was stunning.",

    # Negative reviews
    "The worst experience ever. The food was cold, and the waiter was rude.",
    "I was extremely disappointed with the service. The staff ignored us, and the food was overpriced.",
    "Terrible quality. The place was dirty, and the food tasted awful."
]

# Function to predict sentiment for a list of reviews
def test_reviews_sentiment(model, reviews, tokenizer, max_length=200):
    for review in reviews:
        # Predict sentiment
        sentiment, confidence = predict_sentiment(model, review, tokenizer, max_length)

        # Print results
        print(f"Review: {review}")
        print(f"Predicted Sentiment: {sentiment} (Confidence: {confidence:.4f})")
        print("-" * 50)

# Test the reviews
test_reviews_sentiment(model, test_reviews, tokenizer)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Review: The food was absolutely delicious, and the service was exceptional!
Predicted Sentiment: Positive (Confidence: 0.9865)
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Review: I had a wonderful experience at this restaurant. The ambiance was perfect, and the staff was very friendly.
Predicted Sentiment: Positive (Confidence: 0.9880)
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Review: Highly recommend this place! The dishes were flavorful, and the presentation was stunning.
Predicted Sentiment: Positive (Confidence: 0.9980)
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Review: The worst experience ever. The food was cold, and the waiter was rude.
Predicted Sentiment: Negative (Confidence: 0.9937)
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Review: I was extremely disappoi

In [41]:
import random
import numpy as np

def character_attack(text):
    """Applies character-level perturbations to the given text."""
    def perturb_word(word):
        if len(word) > 3:
            attack_types = ["substitution", "insertion", "spacing"]
            attack = random.choice(attack_types)

            if attack == "substitution":
                # Replace a random character with a similar-looking one
                replacements = {'o': '0', 'e': '3', 'i': '1', 'a': '@', 's': '$', 't': '7'}
                idx = random.randint(0, len(word) - 1)
                word = word[:idx] + replacements.get(word[idx], word[idx]) + word[idx + 1:]

            elif attack == "insertion":
                # Insert a random space in the word
                idx = random.randint(1, len(word) - 1)
                word = word[:idx] + " " + word[idx:]

            elif attack == "spacing":
                # Add unnecessary spacing
                word = " ".join(list(word))

        return word

    words = text.split()
    modified_words = [perturb_word(word) for word in words]
    return " ".join(modified_words)

# Select test samples
num_samples = 5
random_indices = np.random.choice(len(te_x), num_samples, replace=False)

for i in random_indices:
    original_sequence = te_x[i]
    true_label = "Positive" if te_y[i] == 1 else "Negative"

    # Decode to text
    original_review = sequence_to_text(original_sequence, tokenizer)

    # Predict sentiment before attack
    original_sentiment, confidence_before = predict_sentiment(model, original_review, tokenizer)

    # Apply character-level attack
    modified_review = character_attack(original_review)

    # Predict sentiment after attack
    modified_sentiment, confidence_after = predict_sentiment(model, modified_review, tokenizer)

    print(f"Original Review: {original_review}")
    print(f"True Sentiment: {true_label}")
    print(f"Predicted Sentiment Before Attack: {original_sentiment} (Confidence: {confidence_before:.4f})")
    print(f"Modified Review: {modified_review}")
    print(f"Predicted Sentiment After Attack: {modified_sentiment} (Confidence: {confidence_after:.4f})")
    print("-" * 80)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Original Review: the exchange has the biggest inventory of used cds and movies in the city so even if one location doesn't have what you're looking for another location might i bought a bunch of cds here before i started downloading everything and i still occasionally purchase a game or dvd here the prices could stand to be a couple bucks cheaper but otherwise it's all good
True Sentiment: Positive
Predicted Sentiment Before Attack: Positive (Confidence: 0.7864)
Modified Review: the exchange has the b iggest i n v e n t o r y of u s e d cds and m ovies in the c1ty so e v e n if one l o c a t i o n does n't h a v e w h a t you're lookin g for a n o t h e r l o c a t i o n m i g h t i boug ht a bunch of cds h e r e b e f o r e i s7arted downlo@ding e v e r y t h i n g and i s t i l l o c c a s i o n a l l y p u r c h a s e a gam3 or dvd he re the pr ices c0uld stand to be a c o u p l e b ucks c h e a p e r but ot

In [45]:
import random
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define a function to perturb text at the character level
def adversarial_attack(text):
    char_map = {
        'a': '@', 'e': '3', 'i': '1', 'o': '0', 'u': 'v',
        's': '$', 't': '7', 'l': '1', 'c': '(', 'g': '9'
    }
    attacked_text = ''.join(char_map.get(c, c) if random.random() < 0.3 else c for c in text)
    return attacked_text

# Function to predict sentiment of input text
def predict_sentiment(model, text, tokenizer, max_length=200):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(padded_sequence)[0][0]
    sentiment = "Positive" if prediction > 0.5 else "Negative"
    confidence = prediction if sentiment == "Positive" else 1 - prediction
    return sentiment, confidence

# Sample reviews (some positive, some negative)
reviews = [
    "I was extremely disappointed with this laptop. The battery life is terrible, and the screen quality is awful.",
    "I absolutely love this phone! The design is sleek, and the camera takes stunning pictures.",
    "I would never recommend this product. It’s a waste of money and completely useless.",
    "The restaurant had great service but the food was awful."
]

# Process and attack reviews
for i, review in enumerate(reviews):
    # Predict original sentiment
    orig_sentiment, orig_confidence = predict_sentiment(model, review, tokenizer)

    # Apply adversarial attack
    attacked_review = adversarial_attack(review)

    # Predict sentiment after attack
    adv_sentiment, adv_confidence = predict_sentiment(model, attacked_review, tokenizer)

    # Display results
    print(f"Review {i+1}:")
    print(f"Original: {review}")
    print(f"Original Sentiment: {orig_sentiment} (Confidence: {orig_confidence:.4f})")
    print(f"Attacked: {attacked_review}")
    print(f"Attacked Sentiment: {adv_sentiment} (Confidence: {adv_confidence:.4f})")
    print("-" * 80)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Review 1:
Original: I was extremely disappointed with this laptop. The battery life is terrible, and the screen quality is awful.
Original Sentiment: Negative (Confidence: 0.9941)
Attacked: I was extreme1y di$appointed with 7h1s lapt0p. The batt3ry l1f3 is t3rrible, and the scre3n qvality i$ awfu1.
Attacked Sentiment: Positive (Confidence: 0.8003)
--------------------------------------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Review 2:
Original: I absolutely love this phone! The design is sleek, and the camera takes stunning pictures.
Original Sentiment: Positive (Confidence: 0.9500)
Attacked: I abs01vt31y l0ve this phone! The d3si9n i$ $leek, and the camera t@kes s7unning pic7ures.
Attacked Sentiment: Positive (Confidence: 0.9114)
--------------------------------------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━